In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
from snmfem.conf import RESULTS_PATH, DATASETS_PATH
from snmfem.experiments import load_data
from snmfem.measures import find_min_angle, find_min_MSE, spectral_angle
from pathlib import Path
import matplotlib.pyplot as plt
import re
import numpy as np
import hyperspy.api as hs


C:\ProgramData\Anaconda3\lib\site-packages\pyUSID\viz\__init__.py:18: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  FutureWarning)


In [62]:
f1 = hs.load("0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_Brg spectrum.hdf5")
f2 = hs.load("0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_CaPv spectrum.hdf5")
f3 = hs.load("0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_Fp spectrum.hdf5")
scale = f1.axes_manager[0].scale
offset = f1.axes_manager[0].offset
size = f1.axes_manager[0].size
x_true = np.linspace(offset,offset + size*scale, num = size)
x_recons = np.linspace(0.81805,0.01*1918 + 0.81805,num = 1918)

true_spectra = np.array([f1.data,f2.data,f3.data])#,f1.data])

# Load data and ground truth

In [63]:
file = "71GPa_subsolidus_smooth1.npz"
path = RESULTS_PATH / Path(file)
data = np.load(path)
shape_2d = (201,201)
k = 3
GP = data["D"]
A = data["A"]
# losses = data["losses"]
tt = []
tGP = []
for i in range(k) : 
    temp_true = true_spectra[i]
    temp_GP = GP.T[i]
    new_true = temp_true[x_true > np.min(x_recons)]
    new_GP = temp_GP[x_recons < np.max(x_true)]
    tt.append(new_true)
    tGP.append(new_GP)


trunc_true = np.vstack(tt)
trunc_GP = np.vstack(tGP)

angles, true_ind_spec = find_min_angle(trunc_true,trunc_GP,unique = True, get_ind=True)

In [64]:
new_x_true = x_true[x_true > np.min(x_recons)]

## Plot params

In [65]:
fontsize = 30
scale = 15
aspect_ratio = 1.4
marker_list = ["-o","-s","->","-<","-^","-v","-d"]
mark_space = 20


In [70]:
true_ind_spec = [0,1,2]
angles = []
for i,j in enumerate(true_ind_spec) : 
    angles.append(spectral_angle(trunc_true[i],trunc_GP[j]))
labels = ["Bridgmanite", "Ca-perovskite", "Ferropericlase","brg2"]

In [56]:
np.average(A,axis=1)

array([0.01264547, 0.82923097, 0.08667439, 0.07144917])

# Plot spectra and maps

In [71]:
fig, axes = plt.subplots(2,k,figsize = (scale/k * 2 * aspect_ratio,scale))
for i, spectre in enumerate(true_spectra): 
    axes[1,i].plot(new_x_true,trunc_true[i]/np.max(trunc_true[i]),'bo',label='truth',linewidth=4)
    axes[1,i].plot(x_recons,GP.T[true_ind_spec[i]]/np.max(GP.T[true_ind_spec[i]]) ,'r-',label='reconstructed',markersize=3.5)
    axes[1,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
    axes[1,i].set_xlim(0,10)
    axes[1,i].set_xlabel("Energy (keV)",fontsize = fontsize/2)

    im = axes[0,i].imshow(A[true_ind_spec[i]].reshape(shape_2d))#,vmin = 0.0, vmax = 1.0 )
    # axes[0,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    # axes[i,1].set_ylim(0.0,1.0)
    axes[0,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
    axes[0,i].set_title("{}".format(labels[i]),fontsize = fontsize-2)
    axes[1,0].legend()

rows = ["Reconstructed maps","Normalized Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.55, 0.01, 0.32])
fig.colorbar(im,cax=cbar_ax)

# fig.tight_layout()

plt.show()

In [ ]:
np.savetxt("true_spectra.txt",true_spectra)
np.savetxt("GP.txt",GP.T)

In [185]:
fig, axes = plt.subplots(2,k,figsize = (scale/k * 3 * aspect_ratio,scale))
fig.subplots_adjust(wspace=0.01,hspace=0.01)



for i, spectre in enumerate(true_spectra): 
    # axes[1,i].plot(true_spectra[i],'bo',label='truth',linewidth=4)
    # axes[1,i].plot(GP.T[true_ind_spec[i]] ,'r-',label='reconstructed',markersize=3.5)
    # axes[1,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
    # axes[1,i].set_xlim(0,1000)

    axes[1,i].imshow(A[true_ind_spec[i]].reshape(shape_2d),vmin = 0.0, vmax = 1.0 )
    axes[1,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    axes[1,i].tick_params(axis=u'both', which=u'both',length=0)
    axes[1,i].set_xticklabels([])
    axes[1,i].set_yticklabels([])
    # axes[i,1].set_ylim(0.0,1.0)

    im = axes[0,i].imshow(true_maps[i].reshape(shape_2d),vmin = 0.0, vmax = 1.0)
    axes[0,i].set_title("Phase {}".format(i),fontsize = fontsize)
    axes[0,i].tick_params(axis=u'both', which=u'both',length=0)
    axes[0,i].set_xticklabels([])
    axes[0,i].set_yticklabels([])

    # axes[2,0].legend()

rows = ["Truth","Reconstructed","Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.4, 0.02, 0.3])
fig.colorbar(im,cax=cbar_ax)
cbar_ax.tick_params(labelsize=fontsize)

# fig.tight_layout()

plt.show()

NameError: name 'mse' is not defined

# Plot losses

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(4*scale,scale))

names = list(losses.dtype.names)
for j,name in enumerate(names) :
    if re.match(r".*(loss)",name) : 
        axes[0].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[0].set_yscale("log")
        axes[0].set_xscale("log")
        axes[0].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[0].set_xscale("log")
        axes[0].legend(fontsize = fontsize -2)
        axes[0].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(rel)",name) : 
        axes[1].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[1].legend(fontsize = fontsize -2)
        axes[1].set_yscale("log")
        axes[1].set_xscale("log")
        axes[1].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[1].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(ang)",name) :
        axes[2].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[2].legend(fontsize = fontsize -2)
        axes[2].set_xscale("log")
        axes[2].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[2].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(mse)",name) :
        axes[3].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[3].legend(fontsize = fontsize -2)
        axes[3].set_xscale("log")
        axes[3].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[3].set_xlabel("number of iterations",fontsize = fontsize -5)

# Without ground truth

In [48]:
file = "71GPa_subsolidus_smooth1_k4.npz"
path = RESULTS_PATH / Path(file)
data = np.load(path)
shape_2d = (256,256)
k = 4
GP = data["D"]
A = data["A"]
# x = np.linspace(0.59765,0.59765 + 3880*0.005, num = 3880)
x = np.linspace(0.81805,0.81805 + 1918*0.005, num = 1918)
# x = np.linspace(0.6996000000000001 ,0.6996000000000001  + 800*0.01, num = 800)
fontsize = 30
scale = 15
aspect_ratio = 1.4
marker_list = ["-o","-s","->","-<","-^","-v","-d"]
mark_space = 20

In [49]:
fig, axes = plt.subplots(2,k,figsize = (scale/k * 2 * aspect_ratio,scale))
fig.suptitle(file, fontsize=32)
for i in range(k): 
    # axes[1,i].plot(new_x_true,trunc_true[i]/np.max(trunc_true[i]),'bo',label='truth',linewidth=4)
    axes[1,i].plot(x,GP.T[i] ,'ro-',label='reconstructed',markersize=3.5,markevery = mark_space)
    # axes[1,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
#     axes[1,i].set_xlim(0,10)
    axes[1,i].set_xlabel("Energy (keV)",fontsize = fontsize/2)

    im = axes[0,i].imshow(A[i].reshape(shape_2d))#,vmin = 0.0, vmax = 1.0 )
    # axes[0,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    # axes[i,1].set_ylim(0.0,1.0)
    axes[0,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
#     axes[0,i].set_title("{}".format(labels[i]),fontsize = fontsize-2)
    axes[1,0].legend()

rows = ["Reconstructed maps","Normalized Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.55, 0.01, 0.32])
fig.colorbar(im,cax=cbar_ax)

# fig.tight_layout()

plt.show()

In [37]:
Xl = GP@A
Xl_sum = np.sum(Xl,axis = 1)
a_sum = np.sum(a.data,axis=(0,1))

In [47]:
plt.plot(Xl_sum)
plt.plot(a_sum)
plt.plot(a_sum - Xl_sum)

In [50]:
a = hs.load("71GPa_subsolidus.hspy")
a.axes_manager

<Axes manager, axes: (201, 201|1918)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
           width |    201 |      0 |       0 |  0.0017 |     µm 
          height |    201 |      0 |       0 |  0.0017 |     µm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   1918 |        |    0.82 |    0.01 |    keV

In [45]:
a = hs.load("1537_Robin_cropped.hspy")
a.change_dtype("float64")
a.decomposition(True,algorithm = "NMF",output_dimension = 3,solver = "mu",tol = 1e-4, max_iter = 1000)
k=3

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=3
  centre=None
scikit-learn estimator:
NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=1000,
    n_components=3, random_state=None, shuffle=False, solver='mu', tol=0.0001,
    verbose=0)


In [46]:
l = a.get_decomposition_loadings()
f = a.get_decomposition_factors()

In [47]:
fig, axes = plt.subplots(2,k,figsize = (scale/k * 2 * aspect_ratio,scale))
fig.suptitle("HSPY NMF", fontsize=32)
for i in range(k): 
    # axes[1,i].plot(new_x_true,trunc_true[i]/np.max(trunc_true[i]),'bo',label='truth',linewidth=4)
    axes[1,i].plot(x,f.inav[i].data ,'ro-',label='reconstructed',markersize=3.5,markevery = mark_space)
    # axes[1,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
#     axes[1,i].set_xlim(0,10)
    axes[1,i].set_xlabel("Energy (keV)",fontsize = fontsize/2)

    im = axes[0,i].imshow(l.inav[i].data)#,vmin = 0.0, vmax = 1.0 )
    # axes[0,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    # axes[i,1].set_ylim(0.0,1.0)
    axes[0,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
#     axes[0,i].set_title("{}".format(labels[i]),fontsize = fontsize-2)
    axes[1,0].legend()

rows = ["Reconstructed maps","Normalized Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.55, 0.01, 0.32])
fig.colorbar(im,cax=cbar_ax)

# fig.tight_layout()

plt.show()